In [1]:
import ultralytics
from ultralytics import YOLO
from IPython.display import display
from glob import glob
import torch
import os
import shutil
from sklearn.model_selection import train_test_split
import glob
import xml.etree.ElementTree as etree

IMAGE_SIZE = 640

ultralytics.checks()

print(torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print(torch.version.cuda)
print("Number of GPUs available:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print("GPU", i, ":", torch.cuda.get_device_name(i))

print("Home: ", os.getcwd())

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
Setup complete ✅ (16 CPUs, 23.5 GB RAM, 93.4/1006.9 GB disk)


2.2.2+cu118
CUDA Available: True
11.8
Number of GPUs available: 1
GPU 0 : NVIDIA GeForce RTX 3070
Home:  /home/johnny/course/cs4442/final/yolov8


In [2]:
base_dir = './datasets/'
images_folder = base_dir + "images/"
annotations_folder = base_dir + "annotations/"

print("Images directory exists:", os.path.exists(images_folder))
print("Annotations directory exists:", os.path.exists(annotations_folder))

print(os.listdir(images_folder)[:1]) 
print(os.listdir(annotations_folder)[:1]) 

Images directory exists: True
Annotations directory exists: True
['Cars113.png']
['Cars239.xml']


In [3]:
# Define the names of the directories to create.
dirs_to_create = ['train', 'test', 'val']
subdirs_to_create = ['images', 'labels']

for dir_name in dirs_to_create:
    dir_path = os.path.join(base_dir, dir_name)
    
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

for main_dir in dirs_to_create:  
    for subdir in subdirs_to_create:  
        subdir_path = os.path.join(base_dir, main_dir, subdir)
        if not os.path.exists(subdir_path):
            os.makedirs(subdir_path)

created_dirs = os.listdir(base_dir)
created_subdirs = {main_dir: os.listdir(os.path.join(base_dir, main_dir)) for main_dir in dirs_to_create}

display(created_dirs)
display(created_subdirs)

['test', 'train', 'images', 'annotations', 'val']

{'train': ['labels.cache', 'images', 'labels'],
 'test': ['images', 'labels'],
 'val': ['labels.cache', 'images', 'labels']}

In [4]:
image_files = sorted(glob.glob(os.path.join(images_folder, '*.png')))
annotation_files = sorted(glob.glob(os.path.join(annotations_folder, '*.xml')))

train_images, test_images, train_annotations, test_annotations = train_test_split(image_files, annotation_files, test_size=0.2, random_state=42)
train_images, val_images, train_annotations, val_annotations = train_test_split(train_images, train_annotations, test_size=0.2, random_state=42)

def copy_files(files, target_dir):
    for file in files:
        shutil.copy(file, target_dir)

copy_files(train_images, os.path.join(base_dir, 'train', 'images'))
copy_files(val_images, os.path.join(base_dir, 'val', 'images'))
copy_files(test_images, os.path.join(base_dir, 'test', 'images'))

copy_files(train_annotations, os.path.join(base_dir, 'train', 'labels'))
copy_files(val_annotations, os.path.join(base_dir, 'val', 'labels'))
copy_files(test_annotations, os.path.join(base_dir, 'test', 'labels'))

num_files_copied = {
    'train_images': len(os.listdir(os.path.join(base_dir, 'train', 'images'))),
    'val_images': len(os.listdir(os.path.join(base_dir, 'val', 'images'))),
    'test_images': len(os.listdir(os.path.join(base_dir, 'test', 'images'))),
    'train_annotations': len(os.listdir(os.path.join(base_dir, 'train', 'labels'))),
    'val_annotations': len(os.listdir(os.path.join(base_dir, 'val', 'labels'))),
    'test_annotations': len(os.listdir(os.path.join(base_dir, 'test', 'labels')))
}
display(num_files_copied)

{'train_images': 276,
 'val_images': 70,
 'test_images': 87,
 'train_annotations': 552,
 'val_annotations': 140,
 'test_annotations': 174}

In [5]:
def parse(xml):
    tree = etree.parse(xml)
    root = tree.getroot()
    
    xmax = xmin = ymax = ymin = 0

    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    res = []
    for object in root.iter('object'):
        bndbox = object.find('bndbox')
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)
        res.append([width, height, xmin, ymin, xmax, ymax])

    return res

def normalize_for_yolo(original_width, original_height, xmin, ymin, xmax, ymax):
    x_center = (xmin + xmax) / 2 / original_width
    y_center = (ymin + ymax) / 2 / original_height
    width = (xmax - xmin) / original_width
    height = (ymax - ymin) / original_height
    
    return f"0 {x_center} {y_center} {width} {height}\n"

labels = parse('./datasets/annotations/Cars0.xml')
print(labels)
print(normalize_for_yolo(*(parse('./datasets/annotations/Cars0.xml')[0])))

[[500, 268, 226, 125, 419, 173]]
0 0.645 0.5559701492537313 0.386 0.1791044776119403



In [6]:
base_dir = './datasets'
for dir in dirs_to_create:
    annotations_path = os.path.join(base_dir, dir, 'labels')
    xml_files = [f for f in os.listdir(annotations_path) if f.endswith('.xml')]
    
    for xml_file in xml_files:
        xml_path = os.path.join(annotations_path, xml_file)
        labels = parse(xml_path)

        txt_filename = os.path.splitext(xml_file)[0] + '.txt'
        txt_path = os.path.join(annotations_path, txt_filename)
        
        data_str = []
        for label in labels:
            data_str.append(normalize_for_yolo(*label))
        
        with open(txt_path, 'w') as file:
            file.writelines(data_str)

In [7]:
dataset = './data.yaml'

torch.cuda.empty_cache()

backbone = YOLO('./yolov8n.pt')

results = backbone.train(data=dataset, epochs=100, batch=8, imgsz=IMAGE_SIZE, device='0', plots=True, cache=True)

In [11]:
best = YOLO('./runs/detect/train/weights/best.pt')

results = best.val(data=dataset, epochs=100, batch=8, imgsz=IMAGE_SIZE, device=0, plots=True, cache=True)

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /home/johnny/course/cs4442/final/yolov8/datasets/val/labels.cache... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<?, ?it/s]
val: Caching images (0.1GB True): 100%|██████████| 70/70 [00:00<00:00, 699.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:00<00:00, 12.05it/s]
                   all         70         83      0.903      0.785       0.89      0.592
Speed: 0.3ms preprocess, 2.4ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs/detect/val
